# Layer Analytics
This notebook contains code to parse a saved TensorFlow model file and generate a pandas dataframe that contains information about all the layers.

In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
import plotly.express as px

# enable plotly in VS Studio Code
import plotly.io as pio
pio.renderers.default = "notebook_connected"

from workbench.utils.utils import parse_model_name

In [2]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [3]:
#model_name = "mobilenetv1_0.1_224_c3_o3_000"
model_name = "mobilenetv2_0.5_224_c3_o3_000"

In [4]:
parse_model_name(model_name)

'mobilenetv2'

In [15]:
filepath = f"i:/tinyml/tiny_cnn/models/{model_name}/{model_name}_layers.pkl"
df = pd.read_pickle(filepath)

In [16]:
df.columns

Index(['index', 'name', 'layer_type', 'input_shape', 'input', 'output_shape',
       'dtype', 'params', 'variables', 'weights_shape', 'weights_name',
       'weights_dtype', '#_activations', 'OPS', 'MACS', 'filters', 'trainable',
       'kernel_size', 'strides', 'padding', 'dilation_rate', 'groups',
       'activation', 'use_bias', 'data_format'],
      dtype='object')

In [17]:
df.head(20)

,index,name,layer_type,input_shape,input,output_shape,dtype,params,variables,weights_shape,weights_name,weights_dtype,#_activations,OPS,MACS,filters,trainable,kernel_size,strides,padding,dilation_rate,groups,activation,use_bias,data_format
0,0,input_1,<class 'keras.engine.input_layer.InputLayer'>,"[(None, 224, 224, 3)]","KerasTensor(type_spec=TensorSpec(shape=(None, ...","[(None, 224, 224, 3)]",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,conv2d,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 224, 224, 3)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,432,1,"(3, 3, 3, 16)",conv2d/kernel:0,<dtype: 'float32'>,200704,10838016,5419008,16,True,"(3, 3)","(2, 2)",same,"(1, 1)",1,linear,False,channels_last
2,2,conv1_bn,<class 'keras.layers.normalization.batch_norma...,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,64,4,(16),conv1_bn/gamma:0,<dtype: 'float32'>,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,conv1_relu,<class 'keras.layers.activation.relu.ReLU'>,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,block_1_dw_conv,<class 'keras.layers.convolutional.depthwise_c...,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,144,1,"(3, 3, 16, 1)",block_1_dw_conv/depthwise_kernel:0,<dtype: 'float32'>,200704,3612672,1806336,0,True,"(3, 3)","(1, 1)",same,"(1, 1)",1,linear,False,channels_last
5,5,block_1_dw_bn,<class 'keras.layers.normalization.batch_norma...,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,64,4,(16),block_1_dw_bn/gamma:0,<dtype: 'float32'>,0,0,0,0,0,0,0,0,0,0,0,0,0
6,6,block_1_dw_relu,<class 'keras.layers.activation.relu.ReLU'>,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 16)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,7,block_1_compress,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 112, 112, 16)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 8)",float32,128,1,"(1, 1, 16, 8)",block_1_compress/kernel:0,<dtype: 'float32'>,100352,3211264,1605632,8,True,"(1, 1)","(1, 1)",same,"(1, 1)",1,linear,False,channels_last
8,8,block_1_compress_bn,<class 'keras.layers.normalization.batch_norma...,"(None, 112, 112, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 8)",float32,32,4,(8),block_1_compress_bn/gamma:0,<dtype: 'float32'>,0,0,0,0,0,0,0,0,0,0,0,0,0
9,9,block_2_expand,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 112, 112, 8)","KerasTensor(type_spec=TensorSpec(shape=(None, ...","(None, 112, 112, 48)",float32,384,1,"(1, 1, 8, 48)",block_2_expand/kernel:0,<dtype: 'float32'>,602112,9633792,4816896,48,True,"(1, 1)","(1, 1)",valid,"(1, 1)",1,linear,False,channels_last


In [18]:
#df[df.name.str.startswith("add")]

In [19]:
df["input"][25][0].name

'block_2_compress_bn/FusedBatchNormV3:0'

In [20]:

df["output_shape"] = df["output_shape"].map(str)
df["output_shape"] = df["output_shape"].str.strip("[]").replace('\)\,', ')*', regex=True)
df_output = df["output_shape"].str.split("*", expand=True)
df_output

,0
0,"(None, 224, 224, 3)"
1,"(None, 112, 112, 16)"
2,"(None, 112, 112, 16)"
3,"(None, 112, 112, 16)"
4,"(None, 112, 112, 16)"
...,...
150,"(None, 1, 1, 1280)"
151,"(None, 1, 1, 1280)"
152,"(None, 1, 1, 3)"
153,"(None, 1, 3)"


In [32]:
#df["input_shape"][0] =(None, None, None, None)
df["input_shape"] = df["input_shape"].map(str)
df["input_shape"] = df["input_shape"].str.strip("[]").replace('\)\,', ')*', regex=True)
df_input = df["input_shape"].str.split("*", expand=True)
df_input.columns =[f"input_{x+1}" for x in df_input.columns]
no_input_cols = len(df_input.columns)
print(no_input_cols)
df_input


2


,input_1,input_2
0,"(None, 224, 224, 3)",None
1,"(None, 224, 224, 3)",None
2,"(None, 112, 112, 16)",None
3,"(None, 112, 112, 16)",None
4,"(None, 112, 112, 16)",None
...,...,...
150,"(None, 7, 7, 1280)",None
151,"(None, 1, 1, 1280)",None
152,"(None, 1, 1, 1280)",None
153,"(None, 1, 1, 3)",None


In [33]:
def split_tuples(x):
    x = str(x).strip("()")
    x = x.split(",")
    return x

In [34]:
def return_batch_size(x):
    x = str(x).strip("()")
    split_strings = x.split(",")
    #print(split_strings)
    return split_strings[0].strip()

In [35]:
def return_input_height(x):
    #image_size=(img_height, img_width),

    x = str(x).strip("()")
    split_strings = x.split(",")
    if len(split_strings) in [3,4]:
        return split_strings[1].strip()
    else:
        return "None"


In [36]:
def return_input_width(x):
    #image_size=(img_height, img_width),

    x = str(x).strip("()")
    split_strings = x.split(",")
    if len(split_strings) in [4]:
        return split_strings[2].strip()
    else:
        return "None"


In [37]:
def return_channels(x):
    x = str(x).strip("()")
    split_strings = x.split(",")
    #print(split_strings)

    return split_strings[-1].strip()

In [38]:
#df_input["input_1_split"] = df_input["input_1"].apply(split_tuples)

In [39]:
for input in range(1,no_input_cols+1):
    df_input[f"b_i_{input}"]= df_input[f"input_{input}"].apply(return_batch_size)
    df_input[f"h_i_{input}"] = df_input[f"input_{input}"].apply(return_input_height)
    df_input[f"w_i_{input}"] = df_input[f"input_{input}"].apply(return_input_width)
    df_input[f"c_i_{input}"] = df_input[f"input_{input}"].apply(return_channels)

df_input

,input_1,input_2,b_i_1,h_i_1,w_i_1,c_i_1,b_i_2,h_i_2,w_i_2,c_i_2
0,"(None, 224, 224, 3)",None,None,224,224,3,None,None,None,None
1,"(None, 224, 224, 3)",None,None,224,224,3,None,None,None,None
2,"(None, 112, 112, 16)",None,None,112,112,16,None,None,None,None
3,"(None, 112, 112, 16)",None,None,112,112,16,None,None,None,None
4,"(None, 112, 112, 16)",None,None,112,112,16,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
150,"(None, 7, 7, 1280)",None,None,7,7,1280,None,None,None,None
151,"(None, 1, 1, 1280)",None,None,1,1,1280,None,None,None,None
152,"(None, 1, 1, 1280)",None,None,1,1,1280,None,None,None,None
153,"(None, 1, 1, 3)",None,None,1,1,3,None,None,None,None


In [40]:
#df_input.loc[:,["h_i_1", "w_i_1", "c_i_1"]].replace({"NaN": 1, "None": 1}, inplace=True)

In [53]:
#df_input["w_i_2"].replace('None', np.nan, inplace=True)
df_input["b_i_1"].replace('None', np.nan, inplace=True)
df_input["w_i_1"].replace('None', np.nan, inplace=True)
df_input["h_i_1"].replace('None', np.nan, inplace=True)
df_input["c_i_1"].replace('None', np.nan, inplace=True)

df_input["b_i_2"].replace('None', np.nan, inplace=True)
df_input["w_i_2"].replace('None', np.nan, inplace=True)
df_input["h_i_2"].replace('None', np.nan, inplace=True)
df_input["c_i_2"].replace('None', np.nan, inplace=True)
#df_input.fillna(1, inplace=True)

df_input["#i_1_activations"] = df_input["h_i_1"].map(int) * df_input["w_i_1"].map(int) * df_input["c_i_1"].map(int)


In [68]:
def calculate_activations(x):
    h_i = int(x["h_i_2"])#.map(int)
    w_i = int(x["w_i_2"])#.map(int)
    c_i = int(x["c_i_2"])#.map(int)
    out = h_i * w_i * c_i
    if out >1:
        return out
    else:
        return 0

In [69]:
df_input["#i_2_activations"] = df_input.apply(calculate_activations, axis=1)

In [70]:
df_input

,input_1,input_2,b_i_1,h_i_1,w_i_1,c_i_1,b_i_2,h_i_2,w_i_2,c_i_2,#i_1_activations,#i_2_activations
0,"(None, 224, 224, 3)",1,1.0,224,224,3,1,1,1,1,150528,0
1,"(None, 224, 224, 3)",1,1.0,224,224,3,1,1,1,1,150528,0
2,"(None, 112, 112, 16)",1,1.0,112,112,16,1,1,1,1,200704,0
3,"(None, 112, 112, 16)",1,1.0,112,112,16,1,1,1,1,200704,0
4,"(None, 112, 112, 16)",1,1.0,112,112,16,1,1,1,1,200704,0
...,...,...,...,...,...,...,...,...,...,...,...,...
150,"(None, 7, 7, 1280)",1,1.0,7,7,1280,1,1,1,1,62720,0
151,"(None, 1, 1, 1280)",1,1.0,1,1,1280,1,1,1,1,1280,0
152,"(None, 1, 1, 1280)",1,1.0,1,1,1280,1,1,1,1,1280,0
153,"(None, 1, 1, 3)",1,1.0,1,1,3,1,1,1,1,3,0


In [71]:
df_input["#i_2_activations"].value_counts()

0        145
6272       3
12544      2
9408       2
3920       2
37632      1
Name: #i_2_activations, dtype: int64

In [ ]:
def add_dataframe_features(df):
    # clean up the input & output shape of the Input layer
    df["input_shape"].iloc[0] = df["input_shape"].iloc[0][0]
    df["output_shape"].iloc[0] = (None, None, None, None)# df["output_shape"].iloc[0]#[0]
    df[["batch", "h_o", "w_o", "c_o"]] = pd.DataFrame(df["output_shape"].tolist(), index=df.index)
    df[["batch", "h_i", "w_i", "c_i"]] = pd.DataFrame(df["input_1"].tolist(), index=df.index)
    df["#output_activations"] = df["h_o"] * df["w_o"] * df["c_o"]
    df["peak_activations"] = df["h_i"] * df["w_i"] * df["c_i"] + df["h_o"] * df["w_o"] * df["c_o"]
    df["total_params"] = df["params"].cumsum()
    return df

In [ ]:
df_output = df["output_shape"].str.split()

In [ ]:
df["output_shape"].iloc[0]

In [ ]:
#df = add_dataframe_features(df)

In [ ]:
df.head(10)

In [ ]:
df.head()

In [ ]:
fig = px.bar(df, x="name", y= "peak_activations", text_auto=".2s",
    title=f"Peak activations per layer - {model_name}")
fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="MCU constraint", 
              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig.show()

In [ ]:
fig2 = px.bar(df, x="name", y= "h_i", text_auto=".2s",
    title=f"Input height per layer - {model_name}")
#fig2.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange")
fig2.show()